In [66]:
import streamlit as st
import pandas as pd
import sqlite3
import yfinance as yf

In [79]:
def get_names():
    
    conn = sqlite3.connect('database.db')
    produtos = pd.read_sql('SELECT * FROM produtos', conn)
    conn.close()
    
    names = pd.DataFrame()

    for produto in produtos['Produto']:
        ticker = produto + '.SA'
        ticker = yf.Ticker(ticker)
        name = ticker.info.get('longName')
        new_row = pd.DataFrame({'Produto': [produto], 'Nome': [name]})
        names = pd.concat([names, new_row], ignore_index=True)
    return names

In [86]:
conn = sqlite3.connect('database.db')
produtos = pd.read_sql('SELECT * FROM produtos', conn)
conn.close()

tickers = produtos['Produto']
tickers = tickers.to_list()
tickers = [x + '.SA' for x in tickers]

data = yf.download(tickers, period='1d')['Close']
data = data.reset_index()
data = data.melt(id_vars='Date', var_name='Produto', value_name='Cotacao')
data = data.rename(columns={'Date': 'Data Cotacao'})
data['Data Cotacao'] = data['Data Cotacao'].dt.strftime('%Y-%m-%d')
data['Produto'] = data['Produto'].str.replace('.SA', '')

# Fazer o merge com base na coluna 'Produto'
merged = pd.merge(produtos, data, on='Produto', how='left', suffixes=('', '_novo'))

# Atualizar 'Cotacao Atual' e 'Data Cotacao' no DataFrame 'produtos'
produtos['Cotacao Atual'] = merged['Cotacao'].combine_first(merged['Cotacao Atual']).astype(float)
produtos['Data Cotacao'] = merged['Data Cotacao_novo'].combine_first(merged['Data Cotacao'])
produtos['Data Atualizacao'] = pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')

names = get_names()
display(names)
merged2 = pd.merge(produtos, names, on='Produto', how='left', suffixes=('', '_novo'))

produtos['Nome'] = merged2['Nome'].combine_first(merged2['Nome_novo'])



[*********************100%%**********************]  5 of 5 completed
C:\Users\lfvel\AppData\Local\Temp\ipykernel_23068\1337933766.py:20: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  produtos['Cotacao Atual'] = merged['Cotacao'].combine_first(merged['Cotacao Atual']).astype(float)


,Produto,Nome
0,BTLG11,BTG Pactual Logística Fundo de Investimento Im...
1,HGCR11,CSHG Recebiveis Imobiliarios BC Fundo de Inves...
2,HGLG11,Cshg Logistica - Fundo De Investimento Imobili...
3,RBRP11,Fundo Investimento Imobiliario Rbr Properties Fii
4,XPIN11,Xp Industrial Fundo Investimento Imobiliario


In [ ]:

conn = sqlite3.connect('database.db')
produtos.to_sql('cotacoes', conn, if_exists='replace', index=False)
conn.close()

return "Cotacoes atualizadas com sucesso!"